# Промежуточный проект
# Вариант 1: e-commerce 

### Сауткин Н. Аналитика данных, 27 поток.

---

Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:

1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 
2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)
3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)
4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 
5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти [тут](https://vc.ru/s/productstar/134090-chto-takoe-kogortnyy-analiz). (10 баллов)
6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти [тут](https://guillaume-martin.github.io/rfm-segmentation-with-python.html). (35 баллов)

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.

---

# 1. Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests 
from urllib.parse import urlencode 


---

### 1.1 [olist_customers_datase.csv](https://disk.yandex.ru/d/FUi5uSd6BfG_ig) — таблица с уникальными идентификаторами пользователей
 
- customer_id — позаказный идентификатор пользователя
- customer_unique_id —  уникальный идентификатор пользователя  (в идеальных данных не может быть двух пользователей с одинаковым значением, является уникальным айдишником пользователя)
- customer_zip_code_prefix —  почтовый индекс пользователя
- customer_city —  город доставки пользователя
- customer_state —  штат доставки пользователя

In [2]:
# Загружаем таблицу в DataFrame, используя api Яндекс.Диска.

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/FUi5uSd6BfG_ig' 

download_url = requests.get(base_url + urlencode(dict(public_key=public_key)))\
                       .json()['href'] 

download_response = requests.get(download_url) 
df_customers = pd.read_csv(download_url, sep=',') 
df_customers[:5]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [3]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


<div style="border:solid LightGrey 4px; padding: 15px"> 
Судя по описанию столбцов, на каждого уникального пользователя в таблице может встречаться по несколько строк, т.к. ему присваивается новый позаказный идентификатор "customer_id" при создании каждого заказа.

Проверим это, и нет ли дубликатов в customer_id.

In [4]:
df_customers[['customer_id', 'customer_unique_id']].nunique()

customer_id           99441
customer_unique_id    96096
dtype: int64

<div style="border:solid ForestGreen 4px; padding: 15px"> 
    
В данных нет пропущенных значений. 
    
Число уникальных id действительно меньше кол-ва строк, но не намного. Можно предварительно сказать, что:
    
1. кол-во заказов на пользователя в среднем должно быть близкое к 1. Значит, в первом задании в ответ должны попасть практически все пользователи;
    
2. кол-во строк в таблице с пользователями и в таблице с заказами должно совпасть.

---

### 1.2 [olist_orders_dataset.csv](https://disk.yandex.ru/d/t9Li4JOfgxuUrg) —  таблица заказов

- order_id —  уникальный идентификатор заказа (номер чека)
- customer_id —  позаказный идентификатор пользователя
- order_status —  статус заказа
- order_purchase_timestamp —  время создания заказа
- order_approved_at —  время подтверждения оплаты заказа
- order_delivered_carrier_date —  время передачи заказа в логистическую службу
- order_delivered_customer_date —  время доставки заказа
- order_estimated_delivery_date —  обещанная дата доставки


Уникальные статусы заказов в таблице olist_orders_dataset:
* created —  создан
* approved —  подтверждён
* invoiced —  выставлен счёт
* processing —  в процессе сборки заказа
* shipped —  отгружен со склада
* delivered —  доставлен пользователю
* unavailable —  недоступен
* canceled —  отменён


In [5]:
# Загружаем таблицу в DataFrame, используя api Яндекс.Диска.

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/t9Li4JOfgxuUrg' 

download_url = requests.get(base_url + urlencode(dict(public_key=public_key)))\
                       .json()['href'] 

download_response = requests.get(download_url)
df_orders = pd.read_csv(download_url, sep=',') 
df_orders[:5]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [7]:
# приводим столбцы, сожержаште даты и время к нужному типу данных
df_orders = df_orders.astype({'order_purchase_timestamp': 'datetime64',
                              'order_approved_at': 'datetime64',
                              'order_delivered_carrier_date': 'datetime64',
                              'order_delivered_customer_date': 'datetime64',
                              'order_estimated_delivery_date': 'datetime64'})
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


<div style="border:solid LightGrey 4px; padding: 15px"> 
    Проверим, нет ли дубликатов в order_id.

In [8]:
df_orders[df_orders.duplicated()]['order_id'].count()

0

<div style="border:solid ForestGreen 4px; padding: 15px">

Как и ожидалось, кол-во строк совпало с предыдущей таблицей.
    
Во всех столбцах с указанием времени для удобства тип данных сразу заменен на datetime64, поскольку в дальнейших заданиях будет необходимо работать со временем.

В таблице есть пропущенные значения в столбцах с датами. Но это ожидаемо, т.к. на момент выгрузки данных часть заказов должна была находиться на промежуточных этапах. Это подтверждается и кол-вом пропущенных строк - чем дальше этап, тем больше у него пропущено.


---

### 1.3 [olist_order_items_dataset.csv](https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw) —  товарные позиции, входящие в заказы

- order_id —  уникальный идентификатор заказа (номер чека)
- order_item_id —  идентификатор товара внутри одного заказа (не содержит информацию о количестве товаров)
- product_id —  уникальный идентификатор товара (аналог штрихкода)
- seller_id — уникальный идентификатор производителя товара
- shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- price —  цена за единицу товара
- freight_value —  вес товара

In [9]:
# Загружаем таблицу в DataFrame, используя api Яндекс.Диска.

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw' 

download_url = requests.get(base_url + urlencode(dict(public_key=public_key)))\
                       .json()['href'] 

download_response = requests.get(download_url)
df_items = pd.read_csv(download_url, sep=',') 
df_items[:5]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [10]:
# приводим столбцы, сожержаште даты и время к нужному типу данных
df_items = df_items.astype({'shipping_limit_date': 'datetime64'})

In [11]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [12]:
df_items[['order_id', 'product_id']].nunique()

order_id      98666
product_id    32951
dtype: int64

<div style="border:solid ForestGreen 4px; padding: 15px">

Пропущенных значений в таблице нет. 

Кол-во уникальных значений order_id меньше, чем в таблице с заказами. Значит, данная таблица пополняется данными о новом заказе не сразу при его занесении в предыдущую таблицу.


---

# 2. Определение покупки


<div style="border:solid LightGrey 4px; padding: 15px"> 
    В качестве покупки возьмем заказы, по которым получена оплата.
За исключением тех, которые не могут быть доставлены покупателю и оплата по ним должна быть возвращена.
    

То есть, покупкой считается заказ:
- у которого стоит дата подтверждения оплаты;
- не в статусах "отменен" или "недоступен".

Для наглядности в датафрейм с заказами добавим столбец, где будет указано, подтверждена ли оплата.
Затем сформируем сводную таблицу с новым параметром, списком статусов и кол-вом заказов.

In [13]:
df_orders['is_approved'] = df_orders.order_approved_at == df_orders.order_approved_at

In [14]:
pd.pivot_table(data=df_orders, index=['is_approved', 'order_status'], values='order_id', aggfunc='count')

order_id
is_approved order_status          
False       canceled           141
            created              5
            delivered           14
True        approved             2
            canceled           484
            delivered        96464
            invoiced           314
            processing         301
            shipped           1107
            unavailable        609

<div style="border:solid LightGrey 4px; padding: 15px"> 
Есть 14 заказов в статусе "delivered", по которым не проставлена дата подтверждения оплаты.
    
Очевидно, это ошибка данных. Узнать причину появления ошибки возможности нет, а кол-во таких заказов не критично.
Поэтому будет безопасней не учитывать эти заказы в покупках. 

Для удобства дальнейшей работы добавим в датафрейм столбец с информацией, является ли заказ покупкой. Создадим для этого функцию.

In [15]:
def purchase_conditions(s):
    if (s['is_approved'] == False or (s['order_status'] == 'canceled') or (s['order_status'] == 'unavailable')):
        return False
    else:
        return True
df_orders['is_purchased'] = df_orders.apply(purchase_conditions, axis=1)
df_orders[:3]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_approved,is_purchased
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,True,True
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,True,True
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,True,True


<div style="border:solid LightGrey 4px; padding: 15px"> 
Сформируем сводную таблицу с новым параметром, чтобы проверить корректность полученных данных.

In [16]:
pd.pivot_table(data=df_orders, index=['is_purchased', 'is_approved', 'order_status'], values='order_id', aggfunc='count')        .rename(columns = {'order_id':'num_of_orders'})

num_of_orders
is_purchased is_approved order_status               
False        False       canceled                141
                         created                   5
                         delivered                14
             True        canceled                484
                         unavailable             609
True         True        approved                  2
                         delivered             96464
                         invoiced                314
                         processing              301
                         shipped                1107

In [17]:
df_orders.query("is_purchased")['order_id'].count()

98188

<div style="border:solid ForestGreen 4px; padding: 15px">
Параметрам покупки соответствуют 98188 заказов.

---

# 3. Выполнение заданий

---

### 3.1 Сколько у нас пользователей, которые совершили покупку только один раз?


<div style="border:solid LightGrey 4px; padding: 15px"> 
Для решения задачи объединим таблицы customers и orders.

In [18]:
df_task1 = df_orders.merge(df_customers, on='customer_id', how='outer')
df_task1[:3]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_approved,is_purchased,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,True,True,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,True,True,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,True,True,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO



<div style="border:solid LightGrey 4px; padding: 15px"> 
Отфильтруем заказы, которые считаются покупками.

И посчитаем количество customer_unique_id, которые встречаются только 1 раз.

In [19]:
df_task1.query("is_purchased")\
        .groupby('customer_unique_id', as_index = False)\
        .agg({'customer_id': 'count'})\
        .query("customer_id == 1")\
        .agg({'customer_id': 'count'})

customer_id    92087
dtype: int64

<div style="border:solid ForestGreen 4px; padding: 15px"> 
Из 96096 уникальных пользователей 92087 совершили покупку один раз.

---

### 3.2 Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

<div style="border:solid LightGrey 4px; padding: 15px"> 
Исходя из предварительной обработки данных, заказ может оказаться не доставлен по двум причинам:
    
1. товар из заказа недоступен (статус 'unavailable');
2. заказ отменен (статус 'canceled').
    
Но заказ может быть отменен и после выполнения доставки, такие заказы учитывать в статистике будет некорректно.
    
Для решения задачи возьмем подходящие под условия заказы из таблицы orders.    
Добавим столбец с месяцем заказа.    
И затем агрегацией получим нужные данные.

In [20]:
df_task2 = df_orders.query("order_status == 'unavailable' or order_status == 'canceled' and order_delivered_customer_date.isnull()")
df_task2['order_month'] = df_task2.order_purchase_timestamp.dt.to_period('M')
df_task2[:3]

C:\Users\sautk\AppData\Local\Temp\ipykernel_5572\3442024699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_task2['order_month'] = df_task2.order_purchase_timestamp.dt.to_period('M')


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_approved,is_purchased,order_month
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05,True,False,2017-11
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaT,NaT,2018-08-14,True,False,2018-08
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaT,NaT,2018-02-16,True,False,2018-01


In [21]:
df_task2.groupby(['order_status', 'order_month'])\
        .agg({'order_id': 'count'})\
        .groupby('order_status')\
        .agg({'order_id': 'mean'})\
        .rename(columns = {'order_id':'num_of_orders'})

,num_of_orders
order_status,
canceled,25.791667
unavailable,29.000000


<div style="border:solid ForestGreen 4px; padding: 15px"> 
    
В среднем за месяц 25.8 заказов отменяются до доставки покупателю.    
И 29 заказов не доставляются, т.к. заказанных товаров нет в наличии.

---

### 3.3 По каждому товару определить, в какой день недели товар чаще всего покупается.

<div style="border:solid LightGrey 4px; padding: 15px"> 
Для решения задачи объединим таблицы items и orders, чтобы отфильтровать товары из заказов, которые считаются покупками.

Затем добавим в датафрейм столбец с днем недели, когда была совершена покупка.   
И агрегацией получим нужные данные.

In [35]:
df_task3 = df_orders.merge(df_items, on='order_id', how='inner')
df_task3 = df_task3.query("is_purchased")
df_task3['week_day'] = df_task3['order_purchase_timestamp'].dt.day_name()
df_task3[:3]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_approved,is_purchased,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,week_day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,True,True,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,Monday
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,True,True,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,Tuesday
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,True,True,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,Wednesday


In [36]:
df_task3.groupby('product_id', as_index = False)['week_day'].agg(pd.Series.mode)

,product_id,week_day
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday
1,00088930e925c41fd95ebfe695fd2655,Tuesday
2,0009406fd7479715e4bef61dd91f2462,Thursday
3,000b8f95fcb9e0096488278317764d19,"[Friday, Wednesday]"
4,000d9be29b5207b54e86aa1b1ac54872,Tuesday
...,...,...
32721,fff6177642830a9a94a0f2cba5e476d1,"[Saturday, Sunday]"
32722,fff81cc3158d2725c0655ab9ba0f712c,Monday
32723,fff9553ac224cec9d15d49f5a263411f,Friday
32724,fffdb2d0ec8d6a61f0a0a0db3f25b441,Tuesday


---

### 3.4 Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

<div style="border:solid LightGrey 4px; padding: 15px"> 
Для решения задачи объединим таблицы customers и orders.
И отфильтруем заказы, которые считаются покупками.

In [37]:
df_task4 = df_orders.merge(df_customers, on='customer_id', how='outer')
df_task4 = df_task4.query("is_purchased")
df_task4[:3]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_approved,is_purchased,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,True,True,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,True,True,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,True,True,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO


<div style="border:solid LightGrey 4px; padding: 15px"> 
Добавим в таблицу две новые колонки:
    
- месяц покупки
- кол-во недель в месяце

In [38]:
df_task4['purchased_month'] = df_task4.order_purchase_timestamp.dt.to_period('M')
df_task4['weeks_in_month'] = df_task4['order_approved_at'].apply(lambda x: pd.Period(x, freq='M').days_in_month) / 7
df_task4[:3]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_approved,is_purchased,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,purchased_month,weeks_in_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,True,True,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,2017-10,4.428571
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,True,True,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,2018-07,4.428571
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,True,True,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,2018-08,4.428571


<div style="border:solid LightGrey 4px; padding: 15px"> 
Сделаем предварительную агрегацию данных.

In [39]:
df_task4 = df_task4.groupby(['customer_unique_id', 'purchased_month'], as_index = False)\
                   .agg({'order_id': 'count', 'weeks_in_month': 'max'})\
                   .rename(columns = {'order_id':'num_of_orders', 'purchased_month':'month'})
df_task4[:3]

,customer_unique_id,month,num_of_orders,weeks_in_month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,4.428571
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,4.428571
2,0000f46a3911fa3c0805444483337064,2017-03,1,4.428571


<div style="border:solid LightGrey 4px; padding: 15px"> 
Добавим столбец с искомой метрикой и приведем данные в финальный вид.

In [40]:
df_task4['purchases_per_week'] = df_task4.num_of_orders / df_task4.weeks_in_month
df_task4 = df_task4.drop(columns=['num_of_orders','weeks_in_month'])

In [41]:
df_task4

,customer_unique_id,month,purchases_per_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,0.233333
...,...,...,...
96845,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06,0.233333
96846,fffea47cd6d3cc0a88bd621562a9d061,2017-12,0.225806
96847,ffff371b4d645b6ecea244b27531430a,2017-02,0.250000
96848,ffff5962728ec6157033ef9805bacc48,2018-05,0.225806


---

### 3.5 Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. 

<div style="border:solid LightGrey 4px; padding: 15px"> 

Начнем с объединения таблиц customers и orders.
Отфильтруем заказы, которые считаются покупками.   
Для удобства оставим только нужные для задания столбцы, а дату покупки приведем в вид "год-месяц".

In [42]:
df_task5 = df_orders.merge(df_customers, on='customer_id', how='outer')\
                    .query("is_purchased")[['customer_unique_id','order_approved_at']]
df_task5.order_approved_at = df_task5.order_approved_at.dt.to_period('M')
df_task5[:3]

,customer_unique_id,order_approved_at
0,7c396fd4830fd04220f754e42b4e5bff,2017-10
1,af07308b275d755c9edb36a90c618231,2018-07
2,3a653a41f6f9fc3d2a113cf8398680e8,2018-08


<div style="border:solid LightGrey 4px; padding: 15px"> 

Теперь сделаем дополнительную таблицу. Со списком уникальных покупателей, месяцем их первой покупки и месяцем, на который будем считать retention (название будет order_approved_at для удобства на следующем этапе).

In [30]:
customers_task5 = df_task5.groupby('customer_unique_id', as_index = False)\
                          .agg({'order_approved_at': 'min'})\
                          .rename(columns = {'order_approved_at':'first_month'})
customers_task5['retention_month'] = customers_task5.first_month + 2
customers_task5['month'] = customers_task5.first_month.dt.month
customers_task5[:3]

,customer_unique_id,first_month,retention_month,month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-07,5
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,2018-07,5
2,0000f46a3911fa3c0805444483337064,2017-03,2017-05,3


<div style="border:solid LightGrey 4px; padding: 15px"> 

Две таблицы можно объединить, взяв за основу таблицу customers_task5, используя в качестве ключей customer_unique_id и retention_month - order_approved_at.   

Таким образом в таблице с пользователями появится дополнительный столбец order_approved_at. В нем будут стоять значения, если пользователь совершил покупку в месяц, на который мы смотрим удержание.

In [31]:
df_task5 = customers_task5.merge(df_task5, left_on=['customer_unique_id', 'retention_month'], right_on=['customer_unique_id', 'order_approved_at'], how='left')
df_task5[:3]

,customer_unique_id,first_month,retention_month,month,order_approved_at
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-07,5,NaT
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,2018-07,5,NaT
2,0000f46a3911fa3c0805444483337064,2017-03,2017-05,3,NaT


<div style="border:solid LightGrey 4px; padding: 15px"> 

Поскольку пользователи могли совершить более одной покупки в месяц, на который мы смотрим  удержание, в получившейся таблице могли задублироваться уникальные пользователи.   
С помощью агрегации мы уберем лишние строки. А затем сразу можно посчитать новых и удержанных пользователей по месяцам.

In [32]:
df_task5 = df_task5.groupby(['customer_unique_id', 'first_month', 'month'], as_index = False)\
                   .agg({'order_approved_at':'min'})\
                   .groupby('month', as_index = False)\
                   .agg({'customer_unique_id': 'count', 'order_approved_at':'count'})\
                   .rename(columns = {'customer_unique_id':'new_customers', 'order_approved_at':'retained_customers'})

In [33]:
df_task5

,month,new_customers,retained_customers
0,1,7594,28
1,2,8009,29
2,3,9589,32
3,4,8843,24
4,5,10317,34
5,6,9026,27
6,7,9691,12
7,8,10510,15
8,9,4098,20
9,10,4651,13


<div style="border:solid LightGrey 4px; padding: 15px"> 

Теперь мы можем добавить в таблицу retention rate и ответить на вопрос задания.

In [34]:
df_task5['retention_rate'] = df_task5['retained_customers'] / df_task5['new_customers']
df_task5.sort_values('retention_rate', ascending=False)


,month,new_customers,retained_customers,retention_rate
8,9,4098,20,0.004880
10,11,7053,27,0.003828
0,1,7594,28,0.003687
1,2,8009,29,0.003621
2,3,9589,32,0.003337
4,5,10317,34,0.003296
5,6,9026,27,0.002991
9,10,4651,13,0.002795
3,4,8843,24,0.002714
11,12,5592,15,0.002682


<div style="border:solid ForestGreen 4px; padding: 15px"> 
    
Самый высокий показатель retention у когорты пользователей, совершивших первую покупку в сентябре.

---

In [ ]:
'''
Сначала готовим данные:
— отбираем нужные нам столбцы: пользователи, дату заказа;
— из даты заказа достаём месяц и добавляем его в новый столбец month_purchase
'''

df_cohort = df_customers_with_payment_orders[['customer_unique_id', 'order_purchase_timestamp']].copy()

df_cohort['month_purchase'] = df_cohort['order_purchase_timestamp'].dt.to_period('M')

'''
Для когортного анализа нам нужны:
— первый месяц свершения заказа (когорты);
— история совершения заказов по месяцам;
— количество уникальных пользователей, которые совершили заказ в тот или иной месяц.

Формируем когорты, берём минимальный месяц совершения заказа,
считаем по каждой когорте и месяцу совершению заказа количество уникальных пользователей,
фильтруем данные и оставляем только когорты с 2017-01 по 2017-12,
определяем номер месяца с момента совершения первого заказа month_purchase_number

'''


df_cohort['cohort'] = df_cohort.groupby('customer_unique_id')['order_purchase_timestamp'] \
                 .transform('min') \
                 .dt.to_period('M')

df_cohort = df_cohort.groupby(['cohort', 'month_purchase']) \
              .agg(count_customer=('customer_unique_id', 'nunique')) \
              .reset_index(drop=False)

df_cohort = df_cohort.query('(cohort>="2017-01") and (cohort<="2017-12")')

df_cohort['month_purchase_number'] = df_cohort.month_purchase.astype('int') - df_cohort.cohort.astype('int')

'''
Все данные у нас есть, теперь можно начать формировать когортный анализ.

Сначала делаем сводную таблицу,
потом делим количество пользователей с 1 и последующего месяца на количество пользователей в 0 месяц →
→ получаем долю пользователей, которые к нам возвратились в последующих месяцах,
достаём 3-ий месяц с самым большим retention и записываем в переменную cohort_3_month_with_max_retention,
далее стилизуем наш датафрейм с когортным анализом и записываем в переменную df_retention

'''

df_cohort = df_cohort.pivot(index='cohort', columns='month_purchase_number', values='count_customer')

df_retention = df_cohort.div(df_cohort[0], axis=0)

cohort_3_month_with_max_retention = df_retention[3].sort_values(ascending=False).head(1) * 100

df_retention = df_retention.style.format("{:.2%}", na_rep="")
df_retention